This notebook shows an example of inverse kinematics with multiple tasks for the Talos Humanoid robot. We demonstrate an example where the robot reaches for a goal with its right hand using 3 modes:
1. no feet need to stay on the ground
2. keep both feet on the ground
3. keep only the left foot on the ground 


## Requirement
To run these examples you will need to install these two packages:
* [Pinocchio](https://github.com/stack-of-tasks/pinocchio), a rigid body dynamics library (which does all the kinematics computation for us). You can install it directly through conda `conda install pinocchio -c conda-forge` or follow the [installation instructions](https://stack-of-tasks.github.io/pinocchio/download.html#Install_3)
* [Meshcat](https://github.com/rdeits/meshcat-python) which is used for the visualization of the robot. You can install it directly through pip `pip install meshcat`

In [ ]:
import numpy as np
import pinocchio as pin
import os
import time
import meshcat

In [ ]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'talos_reduced.urdf'

# the number of the frames corresponding to limbs of interest (as exposed in Pinocchio)
BASE_LINK_ID = 2
LEFT_FOOT_ID = 16
RIGHT_FOOT_ID = 30
LEFT_HAND_ID = 56
RIGHT_HAND_ID = 90
HEAD_ID = 106

# we load the urdf models with Pinocchio
urdf = package_dirs + urdf_file
robot = pin.RobotWrapper.BuildFromURDF(urdf, package_dirs, root_joint=pin.JointModelFreeFlyer())

# we create the visualization
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
    
viz.loadViewerModel()


# place the robot in its default position and display it
q0 = pin.neutral(robot.model)
viz.display(q0)

# create a ball to visualize the goal
viz.viewer['ball'].set_object(meshcat.geometry.Sphere(0.05), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))

## inverse kinematics for the hand and feet with priority

Below we show how to move the right hand while keeping both feet on the ground, i.e. the hand will move in the nullspace of the feet and then we will optimize a desired posture for the robot in the nullspace of both the hand and the feet.

You can select 3 different modes:
1. no feet need to stay on the ground
2. keep both feet on the ground
3. keep only the left foot on the ground 

Change `select_mode` to 1, 2 or 3 to select these options

In [ ]:
# inverse kinematics
select_mode = 1

# this is the desired right hand position (no orientation)
right_hand_desired = np.array([0.8,0.0, 0.1])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(right_hand_desired))

# the desired robot posture (its default posture)
robot_desired_posture = pin.neutral(robot.model)

# we start with a random guess for the joint positions (q) 
# here we use the default robot position
q = pin.neutral(robot.model)
viz.display(q)

## we iterate up to max_iterations
max_iterations = 200
for i in range(max_iterations):
    # compute the forward kinematics
    robot.forwardKinematics(q)
    
    # get the position of the feet and right hand
    T_S_RF = robot.framePlacement(q, RIGHT_FOOT_ID)
    T_S_LF = robot.framePlacement(q, LEFT_FOOT_ID)
    T_S_RH = robot.framePlacement(q, RIGHT_HAND_ID)
    
    # get the Jacobian in the right frames
    J_RF = robot.computeFrameJacobian(q, RIGHT_FOOT_ID) # body
    J_LF = robot.computeFrameJacobian(q, LEFT_FOOT_ID) # body
    J_RH = T_S_RH.rotation @ robot.computeFrameJacobian(q, RIGHT_HAND_ID)[:3,:] # body oriented like spatial
    
    epsilon = 10e-4
    
    if select_mode == 1: # no legs need to stay on the ground
        N_Jlegs = np.eye(robot.model.nv)
    else:
        if select_mode == 2:
            # stack the jacobians for the legs 
            J_legs = np.vstack((J_RF, J_LF))
        elif select_mode == 3:
            # use only yhe left leg Jacobian
            J_legs = J_LF
        else:
            print('ERROR this mode does not exist')
            raise
        # compute the pseudo-inverse of the leg(s) and the associated nullspace projector
        pinv_Jlegs = J_legs.T.dot(np.linalg.inv(J_legs.dot(J_legs.T) + epsilon*np.eye(J_legs.shape[0])))
        N_Jlegs = np.eye(robot.model.nv) - pinv_Jlegs.dot(J_legs)
    
    # compute the null-space compatible Jacobian of the hand and the associated pseudo inverse
    J_null_RH = J_RH.dot(N_Jlegs)
    pinv_JnullRH = J_null_RH.T.dot(np.linalg.inv(J_null_RH.dot(J_null_RH.T) + epsilon*np.eye(3)))
    
    # compute the posture Jacobian (nullspace compatible)
    N_task = N_Jlegs.dot(np.eye(robot.model.nv) - pinv_JnullRH.dot(J_null_RH))
    
    # Note that for the leg we just want the leg to not move (i.e. desired velocity of 0)
    # so there is no need to compute an error - since it is dx = 0 and J+ dx = 0 too so we omit it
    
    # compute the error of the posture task / we need to treat base pose and joint position errors differently
    # because the base pose contains orientations so we need a log and q contains a quaternion
    posture_error = np.zeros([robot.model.nv])
    posture_error[:6] = pin.log(pin.XYZQUATToSE3(q[:7]).actInv(pin.XYZQUATToSE3(q_desired[:7]))).vector
    posture_error[6:] = 10.*(q_desired[7:] - q[7:])
    
    # compute the right hand error
    gain = 5.
    err = gain * (right_hand_desired - T_S_RH.translation)
    
    # compute do one step of diff. IK
    alpha = 0.01
    dq = pinv_JnullRH.dot(err) + N_task.dot(posture_error)
    # we cannot update q by just adding dq because we have a quaternion (orientation of the robot)
    # so we use the integrate function of Pinocchio which does proper update on SE(3)
    q = pin.integrate(robot.model, q, dq*alpha)
    
    # skipped error checks - to be added - we just iterate until the end
    
    # we display the new guess (and sleep to make it nicely visible with rendering)
    # turn this off to see how fast this really is
    viz.display(q)
    time.sleep(0.01)
    
print(f'we found joint positions:\n {q}')
print(f'this gives an end-effector position of {T_S_RH.translation}')
print(f'the desired right hand position was {right_hand_desired}')